## Detector Efficiency for an $\alpha$ particle emitted from Polonium

This program calculates the average background activity and the net activity of a radionuclide (with the background subtracted). It then compares the measured activity with the current activity from a radionuclide with a known half-life corrected for the solid angle subtended by the 
#radionuclide at the detector with diameter,d, placed a distance, l, away. It assumes that the exact date the radionuclide was synthesized and  
#its initial activity are known. The result is the calculation of the detector efficiency.

### Measured counts for background and radionuclide

In [17]:
import numpy as np
import pandas as pd
from datetime import datetime
#read time interval versus background count and Sr 90 emissions
test_df_in = pd.read_excel('Sr90_test.xlsx', dtype ={'delta_t': int, 'counts': int})
#print(test_df_in)# prints column headers, row index number, delta_t and counts. 
#If you do not want to print the column header and the row index, then use test_df_in.values
#print(test_df_in.loc[6])
#Important: Your raw data csv or xlsx file should have two columns - Column 0 - time interval; Column 1: counts. Refer to the sample xlsx file Sr90_test.xlsx 
#to know how the raw data is presented.
#When background data acquisition is complete, create a new row with integers -1 typed in both columns. This indicates the end of the background data.
#Proceed to measure the activity of a radionuclide. When you have collected data over sufficient number of time intervals, type -10 in both columns to 
#indicate conclusion of radionuclide activity.
#measure the background activity
#initialize
i = 0
delta_t_bg = (test_df_in.iloc[0,0])
nbg_sum = 0
T_tot_bgnd = 0
bg_index = 1
#print (delta_t_bg)#interval over which background radiation is counted. This is done over several intervals designated as bg_index
while (i >= 0):
    if (test_df_in.iloc[i,0] != -1) and (test_df_in.iloc[i,1] != -1): 
        nbg_sum = nbg_sum + test_df_in.iloc[i,1]#cumulate background counts
        i += 1   
    elif (test_df_in.iloc[i,0] == -1) and (test_df_in.iloc[i,1] == -1):#if the limiting number -1 is encountered in the two columns, stop cumulating backgroud.
#exit loop and proceed to counting emissions from radionuclide
        print('background calculated') 
        break
bg_index = i
T_tot_bgnd = i*delta_t_bg
nbg_avg = nbg_sum / T_tot_bgnd #average background activity
print (bg_index, T_tot_bgnd, nbg_sum, nbg_avg)

#activity of a radionuclide
N_T = 0 #initialize
bg_index_nuc = bg_index+1
delta_t_nuc = (test_df_in.iloc[bg_index_nuc,0])#time interval over which radionuclide emissions are counted. This is done total of bg_index_nuc times.
#print (delta_t_nuc)
i = bg_index_nuc
#print (i)
while(i >= bg_index_nuc):
    if (test_df_in.iloc[i,0] != -10) and (test_df_in.iloc[i,1] != -10):
        N_T = N_T + test_df_in.iloc[i,1]#cumulate counts from adionuclide over a time interval delta_t_nuc
        i += 1
    elif (test_df_in.iloc[i,0] == -10) and (test_df_in.iloc[i,1] == -10):
        print('average radionuclide activity calculated')
        break
bg_index_f = i
#print (bg_index_f)
N_index = bg_index_f - (bg_index_nuc)
T_tot_nuc = N_index*delta_t_nuc #total duration, in seconds, where radioactive emission is measured
R_T = N_T / T_tot_nuc #total measured activity including background, in counts/sec
R_S = R_T - nbg_avg #Activity (net), i.e background subtracted, in counts/sec
print (N_index, T_tot_nuc, N_T, R_S)#check to ensure every data point with radionuclide is included

background calculated
40 400 134 0.335
average radionuclide activity calculated
20 200 37470 187.015


### Calculate current activity from the known activity on a reference starting day.

In [18]:
di = '2005/1/1'#date of synthesis stamped on the radionuclide. Change this date to that for your sample
df = '2005/3/21'#date on which you acquired the data.
di = datetime.strptime(di,"%Y/%m/%d")
df = datetime.strptime(df,"%Y/%m/%d")
timedelta = (abs(df-di))#difference in days
timediff = timedelta.days
print("Elapsed time", timediff, "days")
t_half = 138 #half life of a radionuclide in days. Enter the value in days for the radionuclide you are using
decay_const = np.log(2)/t_half #/day
A_init = 0.1 #initial activity in microcurie
A_S = (A_init*1e-6*3.7e10)*np.exp(-decay_const*timediff)#decays per sec
A_S = round(A_S,-1)#round the result to nearest 10s
print("Current Activity in counts/s", A_S)

Elapsed time 79 days
Current Activity in counts/s 2490.0


### Correct for area of the detector

In [19]:
import math
l =  10 #distance between radionuclide and GM counter. l=10 mm is used as an example
d = 31 #diameter of the mica window in mm. 35 mm is used as an example.
#geometric ratio, GR
GR = 0.5- (l/(2*math.sqrt(l**2+((d/2)**2))))
print("geometric ratio", GR)
A_Scorr = A_S*GR
A_Scorr = round(A_Scorr,-1)
print("geometry corrected counts", A_Scorr)


geometric ratio 0.22893656508895344
geometry corrected counts 570.0


### Calculate detector efficiency for an alpha particle

In [20]:
detect_eff = R_S/A_Scorr
print ("detector efficiency for alpha particle is", detect_eff)

detector efficiency for alpha particle 0.32809649122807016
